# Batch Correlation Regularizer

In [1]:
import sys
sys.path.append('..')

In [2]:
from keras_bcr import BatchCorrRegularizer
import tensorflow as tf

class MyResNetBlock(tf.keras.layers.Layer):
    def __init__(self, n_units=128, bcr_rate=1e-6):
        super(MyResNetBlock, self).__init__()
        self.linear = tf.keras.layers.Dense(n_units)
        self.normal = tf.keras.layers.BatchNormalization()
        self.activ = tf.keras.layers.Activation('gelu')
        self.bcr = BatchCorrRegularizer(bcr_rate)
        self.add = tf.keras.layers.Add()
    def call(self, inputs):
        h = self.linear(inputs)
        h = self.normal(h)
        h = self.activ(h)
        h = self.bcr([h, inputs])  # before Add
        h = self.add([h, inputs])
        return h

In [3]:
n_features = 128

model = tf.keras.Sequential([
    MyResNetBlock(n_units=n_features, bcr_rate=1.),
    tf.keras.layers.Dense(1)
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.losses.MeanSquaredError()]
)

2022-05-27 09:49:56.994384: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
n_sample_size = 1000
# X = tf.random.uniform((n_sample_size, n_features))  
# y = tf.random.uniform((n_sample_size,))  

seqlen = 64
X = tf.random.uniform((n_sample_size, seqlen, n_features))  
y = tf.random.uniform((n_sample_size, seqlen))  

In [5]:
model.fit(X, y, epochs=10)

Epoch 1/10
32/32 [==============================] - 3s 25ms/step - loss: 1.0546 - mean_squared_error: 0.8841 - batch_corr_regularizer: 0.1622
Epoch 2/10
32/32 [==============================] - 1s 29ms/step - loss: 0.4504 - mean_squared_error: 0.2923 - batch_corr_regularizer: 0.1608
Epoch 3/10
32/32 [==============================] - 1s 22ms/step - loss: 0.3940 - mean_squared_error: 0.2376 - batch_corr_regularizer: 0.1597
Epoch 4/10
32/32 [==============================] - 0s 15ms/step - loss: 0.3754 - mean_squared_error: 0.2206 - batch_corr_regularizer: 0.1582
Epoch 5/10
32/32 [==============================] - 0s 14ms/step - loss: 0.3634 - mean_squared_error: 0.2096 - batch_corr_regularizer: 0.1575
Epoch 6/10
32/32 [==============================] - 1s 16ms/step - loss: 0.3530 - mean_squared_error: 0.1998 - batch_corr_regularizer: 0.1567
Epoch 7/10
32/32 [==============================] - 0s 14ms/step - loss: 0.3432 - mean_squared_error: 0.1910 - batch_corr_regularizer: 0.1562
Epoch 